<a href="https://colab.research.google.com/github/Shyam-Khokhariya/ML-Python/blob/master/C16_Movie_Review_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install pyprind
import pyprind
import pandas as pd
import numpy as np
import re
from string import punctuation

In [0]:
df=pd.read_csv("movie_data.csv",encoding="utf-8")

In [13]:
review="Hello, everyone . My Self ! @ shyam ?"
text=""
for c in review:
  if c not in punctuation:
    text=text+c
  else:
    text=text+" "
print(text)
text=" ".join(text.split()).lower()
print(text)

sequence_length=200
sequences=np.zeros(sequence_length,dtype=str)
r_arr=np.array(text)
sequences[-len(text):]=text[-sequence_length:]
print(sequences)

SyntaxError: ignored

In [16]:
from collections import Counter
counts = Counter()
pbar = pyprind.ProgBar(len(df['review']),title='Counting words occurrences')
for i,review in enumerate(df['review']):
  #text = ''.join([c if c not in punctuation else ' '+c+' ' for c in review]).lower()
  text=""
  for c in review:
    if c not in punctuation:
      text=text+c
    else:
      text=text+" "
  text=" ".join(text.split()).lower()
  df.loc[i,'review'] = text
  pbar.update()
  counts.update(text.split())
 ## Create a mapping
 ## Map each unique word to an integer
word_counts = sorted(counts, key=counts.get, reverse=True)
print(word_counts[:5])
word_to_int = {word: ii for ii, word in  enumerate(word_counts, 1)}
mapped_reviews = []
pbar = pyprind.ProgBar(len(df['review']),  title='Map reviews to ints')
for review in df['review']:
  mapped_reviews.append([word_to_int[word] for word in review.split()])
  pbar.update()

Counting words occurrences
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:03:58
Map reviews to ints


['the', 'and', 'a', 'of', 'to']


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


In [0]:
#Defining same length sequences

sequence_length=200
sequences=np.zeros((len(mapped_reviews),sequence_length),dtype=int)
for i,row in enumerate(mapped_reviews):
  review_arr=np.array(row)
  sequences[i,-len(row):]=review_arr[-sequence_length:]


In [0]:
X_train=sequences[:25000,:]
y_train=df.loc[:25000,"sentiment"].values
X_test=sequences[25000:,:]
y_test=df.loc[25000:,"sentiment"].values

In [0]:
np.random.seed(123)

#Function for mini-batch
def create_batch_generator(x,y=None,batch_size=64):
  n_batches=len(x)//batch_size
  x=x[:n_batches*batch_size]
  if y is not None:
    y=y[:n_batches*batch_size]
  for ii in range(0,len(x),batch_size):
    if y is not None:
      yield x[ii:ii+batch_size],y[ii:ii+batch_size]
    else:
      yield x[ii:ii+batch_size]
      

In [0]:
embedding=tf.Variable(tf.random_uniform(shape=(n_words,embedding_size),minval=-1,maxval=1))
embed_x=tf.nn.embedding_lookup(embedding,tf_x)